## Import Packages and Data Loadings

In [1]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# !unzip data.zip

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-11-20 01:31:38.830386: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Model Creation

In [3]:
# Method to create a CNN model
def createModel():
    # Create the CNN model
    model = Sequential()

    # Input layer
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))

    # Max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten layer
    model.add(Flatten())

    # Dense layer with 64 neurons
    model.add(Dense(64, activation='relu'))

    # Output layer with 1 neuron for binary classification
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
# Method to create an optimizer and compile with that CNN model
def initOptimizer(model):
    # Compile the model with SGD optimizer
    sgd_optimizer = SGD(lr=0.002, momentum=0.8)
    model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# Display the model summary
model = createModel()
model = initOptimizer(model)
model.summary()

2023-11-20 01:31:40.644120: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-20 01:31:40.651218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-20 01:31:40.651809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-20 01:31:40.652721: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


### Question 1

Since we have a binary classification problem, what is the best loss function for us?

    mean squared error
    binary crossentropy
    categorical crossentropy
    cosine similarity

    Note: since we specify an activation for the output layer, we don't need to set from_logits=True


### Answer: binary crossentropy



### Question 2

What's the number of parameters in the convolutional layer of our model? You can use the summary method for that.

    1
    65
    896
    11214912
    
### Answer: 11214912

In [6]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define the train and test generators
train_generator = train_datagen.flow_from_directory(
    './data/train/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    './data/test/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

# Train the model and collect metrics
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

# Calculate median of training accuracy and standard deviation of training loss
median_train_accuracy = np.median(history.history['accuracy'])
std_dev_train_loss = np.std(history.history['loss'])

print(f"Median of training accuracy: {median_train_accuracy}")
print(f"Standard deviation of training loss: {std_dev_train_loss}")

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.
Epoch 1/10


2023-11-20 01:31:42.846804: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-11-20 01:31:43.488294: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-20 01:31:43.488972: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-20 01:31:43.489007: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-11-20 01:31:43.489479: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-20 01:31:43.489551: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


184/184 [==============================] - 11s 48ms/step - loss: 0.6604 - accuracy: 0.5953 - val_loss: 0.5968 - val_accuracy: 0.6765
Epoch 2/10
184/184 [==============================] - 9s 47ms/step - loss: 0.5903 - accuracy: 0.6908 - val_loss: 0.5663 - val_accuracy: 0.6961
Epoch 3/10
184/184 [==============================] - 9s 47ms/step - loss: 0.5501 - accuracy: 0.7278 - val_loss: 0.5369 - val_accuracy: 0.7473
Epoch 4/10
184/184 [==============================] - 9s 47ms/step - loss: 0.5089 - accuracy: 0.7648 - val_loss: 0.5225 - val_accuracy: 0.7636
Epoch 5/10
184/184 [==============================] - 9s 47ms/step - loss: 0.4833 - accuracy: 0.7716 - val_loss: 0.5143 - val_accuracy: 0.7505
Epoch 6/10
184/184 [==============================] - 9s 47ms/step - loss: 0.4711 - accuracy: 0.7884 - val_loss: 0.5118 - val_accuracy: 0.7397
Epoch 7/10
184/184 [==============================] - 9s 47ms/step - loss: 0.4461 - accuracy: 0.8123 - val_loss: 0.4862 - val_accuracy: 0.7734
Epoch 8/1

In [7]:
print(f"Median of training accuracy: {round(median_train_accuracy,2)}")
print(f"Standard deviation of training loss: {round(std_dev_train_loss,3)}")

Median of training accuracy: 0.78
Standard deviation of training loss: 0.088


### Question 3

What is the median of training accuracy for all the epochs for this model?

    0.20
    0.40
    0.60
    0.80

### Answer:  0.80

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

    0.031
    0.061
    0.091
    0.131
    
### Answer: 0.091

In [8]:
# Data generators with augmentations
train_datagen_augmented = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen_augmented = ImageDataGenerator(rescale=1./255)

# Define the train and test generators
train_generator = train_datagen.flow_from_directory(
    './data/train/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    './data/test/',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

# Continue training with data augmentations
history_augmented = model.fit(
    train_generator,  # Use the generator with augmentations
    epochs=20,        # Train for 10 more epochs
    validation_data=test_generator,
    initial_epoch=10  # Start from the last epoch of the previous training
)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.
Epoch 11/20
184/184 [==============================] - 9s 47ms/step - loss: 0.3335 - accuracy: 0.8640 - val_loss: 0.5329 - val_accuracy: 0.7582
Epoch 12/20
184/184 [==============================] - 9s 47ms/step - loss: 0.2899 - accuracy: 0.8975 - val_loss: 0.6221 - val_accuracy: 0.7059
Epoch 13/20
184/184 [==============================] - 9s 46ms/step - loss: 0.2558 - accuracy: 0.9094 - val_loss: 0.5139 - val_accuracy: 0.7658
Epoch 14/20
184/184 [==============================] - 9s 47ms/step - loss: 0.2189 - accuracy: 0.9285 - val_loss: 0.5394 - val_accuracy: 0.7658
Epoch 15/20
184/184 [==============================] - 9s 47ms/step - loss: 0.1868 - accuracy: 0.9426 - val_loss: 0.5311 - val_accuracy: 0.7636
Epoch 16/20
184/184 [==============================] - 9s 47ms/step - loss: 0.1515 - accuracy: 0.9578 - val_loss: 0.5862 - val_accuracy: 0.7571
Epoch 17/20
184/184 [==============================] 

In [9]:
# Calculate median of training accuracy and standard deviation of training loss
median_train_accuracy_aug = np.median(history_augmented.history['accuracy'])
std_dev_train_loss_aug = np.std(history_augmented.history['loss'])

print(f"Median of training accuracy: {median_train_accuracy_aug}")
print(f"Standard deviation of training loss: {std_dev_train_loss_aug}")

Median of training accuracy: 0.9502311646938324
Standard deviation of training loss: 0.08548457320807519


In [10]:
# Calculate the mean of test loss for all epochs
mean_test_loss_augmented = np.mean(history_augmented.history['val_loss'])

# Calculate the average of test accuracy for the last 5 epochs
avg_test_accuracy_last_5_epochs_augmented = np.mean(history_augmented.history['val_accuracy'][-5:])

print(f"Mean of test loss for all epochs with augmentations: {round(mean_test_loss_augmented,2)}")
print(f"Average of test accuracy for the last 5 epochs with augmentations: {round(avg_test_accuracy_last_5_epochs_augmented,2)}")

Mean of test loss for all epochs with augmentations: 0.59
Average of test accuracy for the last 5 epochs with augmentations: 0.77


### Question 5

Let's train our model for 10 more epochs using the same code as previously.

    Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

    0.18
    0.48
    0.78
    0.108

### Answer: 0.48

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

    0.38
    0.58
    0.78
    0.98

### Answer: 0.78